In [ ]:
%reset -f
# math stuff
import sympy as sym
import numpy as np 

from sympy import sin, cos, atan, pi

#for animations 
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

In [ ]:
# symbolic variables
x1,z1,th1 = sym.symbols(['x1','z1','th1'])

dx1b,dz1b,dth1 = sym.symbols(['dx1b','dz1b','dth1b'])

ddx1b,ddz1b,ddth1 = sym.symbols(['ddx1b','ddz1b','ddth1b'])

de, df, T, V, a = sym.symbols(['de','df','T','V','a'])
m1 = sym.symbols('m1')
I1 = sym.symbols('I1')

S, b, c, A, e = sym.symbols(['S','b','c','A','e'])

CL_0,CL_a,CL_Q,CL_de, CL_df = sym.symbols(['C_L_0','C_L_alpha','C_L_Q','C_L_delta_E', 'CL_df'])

CD_0 = sym.symbols('C_D_0')

Cm_0,Cm_a,Cm_Q,Cm_de, Cm_df = sym.symbols(['C_m_0','C_m_alpha','C_m_Q','C_m_delta_E','Cm_df'])

rho = sym.symbols('rho')

g = sym.symbols('g')

In [ ]:
# Coefficient of lift
CL= CL_0 + CL_a*a + CL_de*de + CL_df*df
# Coefficient of drag
CD = CD_0 + (CL*CL)/(sym.pi*A*e)
Q = dth1
# X-force coefficient in wind (stability) axis
CXs = -CD
# Z-force coefficient in wind (stability) axis
CZs = -CL + (c/(2*V))*(CL_Q*Q)
# Pitch moment coefficient in wind (stability) axis
Cms = Cm_0 + Cm_a*a + (c/(2*V))*(Cm_Q*Q) + Cm_de*de + Cm_df*df

# X-force coefficient in body axis
CX = CXs*sym.cos(a) - CZs*sym.sin(a)
# Z-force coefficient in body axis
CZ = CZs*sym.cos(a) + CXs*sym.sin(a)
# Pitch moment coefficient in body axis
Cm = Cms

In [ ]:
# forces
# X-force in body axis
Xa = 0.5*rho*V*V*S*CX
# Z-force in body axis
Za = 0.5*rho*V*V*S*CZ
# Pitch moment in body axis
Ma = 0.5*rho*V*V*S*c*Cm

X = Xa + T - sym.sin(th1)*(m1*g)
Z = Za + sym.cos(th1)*(m1*g)
M = Ma

In [ ]:
# equations of motion
dQ = ddth1
EOM1 = sym.simplify(ddx1b - (1/m1 * X - dz1b*Q))
EOM2 = sym.simplify(ddz1b - (1/m1 * Z + dx1b*Q))
EOM3 = sym.simplify(dQ - M/I1)

In [ ]:
# import pickle as pkl
# data = {"EOM1":EOM1,
#         "EOM2":EOM2,
#         "EOM3":EOM3,
#        }

# outfile = open('../../Optimization/UAV_EOM','wb')
# pkl.dump(data,outfile)
# outfile.close()

In [ ]:
#lambdify
func_map = {'sin':sin, 
            'cos':cos}
sym_list = [
    x1,z1,th1,
    dx1b, dz1b, dth1,
    ddx1b, ddz1b, ddth1,
    V, a, T,
    de, df,
    m1, I1,
    S, b, c, A, e,
    CD_0,
    CL_0,CL_a,CL_Q,CL_de,CL_df,
    Cm_0,Cm_a,Cm_Q,Cm_de,Cm_df,
    rho, g, sym.pi   
]

lambEOM1 = sym.lambdify(sym_list, EOM1, modules = [func_map])
lambEOM2 = sym.lambdify(sym_list,EOM2,modules = [func_map])
lambEOM3 = sym.lambdify(sym_list,EOM3,modules = [func_map])

In [ ]:
g_ =  9.81

m_ =  6.35

Iy_ =  0.514

# paramters of the aircraft
S_ =  0.6975 #0.5017)
b_ =   1.918 #1.73)
c_ =   0.363 #0.2993)
A_ =  5.28 #5.9655)
e_ =  0.858 #0.85)

# aerodynamic stuff
CL_0_ =  0.243200  #0.0)
CD_0_ =  0.12 #0.07)
Cm_0_ =  -0.026700 #0.0)

CL_a_ =  4.240906 #5.1309)
Cm_a_ =  -0.780993 #-0.2954)

CL_Q_ =  7.046092 #7.7330)
Cm_Q_ =  -7.220962 #-10.2807)

CL_de_ =  0.419064 #0.71266)
Cm_de_ =  -0.922107 #-1.5853)

CL_df_ =  0.936323
Cm_df_ =  0.111822

rho_ =1.225 # air dencity"""

In [ ]:
N = 100
h = 0.02

# initual conditions
Q_0 = 0*np.pi/180

xb_0, zb_0 = 0.0,0.0
x_0, z_0 = 0.0,0.0

# calculate the trim conditions
VT = 18.0
qT = 0.5 * rho * VT * VT;
trim_angles = (sym.Matrix([[CL_a,CL_de],[Cm_a,Cm_de]]))**-1 * sym.Matrix(([[m1*g/(qT*S)-CL_0],[-Cm_0]]))

values = [(CL_a,CL_a_),(CL_de,CL_de_),(Cm_a,Cm_a_),
          (Cm_de,Cm_de_),(S,S_),(rho,rho_ ),(m1, m_),(g, g_),
         (Cm_0, Cm_0_), (CL_0, CL_0_), (Cm_df, Cm_df_), (CL_df, CL_df_),(CD_0,CD_0)]

trim_angles = trim_angles.subs(values)
# α_trim = θ_trim
α_t = float(trim_angles[0]) #*pi/180
δ_e = float(trim_angles[1]) - 40*pi/180
θ_t = float(trim_angles[0]) #*pi/180         #<---- just for now 

# initual velocity
U_t, W_t = VT*cos(θ_t),VT*sin(θ_t)
Ndot_t, Ddot_t = cos(θ_t)*U_t + sin(θ_t)*W_t, -sin(θ_t)*U_t + cos(θ_t)*W_t;

CLt = (m_*g_)/(qT*S_)
CDt = CD_0 + (CLt*CLt)/(pi*A*e)

T_t = qT*S*CDt*cos(α_t) - qT*S*CLt*sin(α_t) + m_*g_*sin(α_t);

params = [(T,T_t),(de, δ_e),(g, g_),(m1, m_),
          (CL_a,CL_a_),(CL_de,CL_de_),(Cm_a,Cm_a_),
          (Cm_de,Cm_de_),(S,S_),(rho,rho_ ),
          (Cm_0, Cm_0_), (CL_0, CL_0_), (CD_0, CD_0_), (A, A_), (e, e_), 
          (CL_df, CL_df_), (Cm_df,Cm_df_),(df, 0),(I1,Iy_),(c,c_),
          (Cm_Q, Cm_Q_),(CL_Q, CL_Q_)]

T_t = T_t.subs(params)
# arrays
U_ar, W_ar = [float(U_t)], [float(W_t)]
Ndot_ar, Ddot_ar = [Ndot_t], [Ddot_t]
N_ar, D_ar = [x_0],[z_0]

Q_ar = [0]
θ_ar = [θ_t]  #<----- for now , i think"""

In [ ]:
for i in range(1,N):
    
    if U_ar[i-1] != 0:
        α = atan(W_ar[i-1]/U_ar[i-1])
    else:
        α = 0
    V_ = (U_ar[i-1]**2 + W_ar[i-1]**2)**0.5
    
    #past = [(dx1b, U_ar[i-1]),(dz1b, W_ar[i-1]),(dth1, Q_ar[i-1]),(a, α),(V, V_),(th1, θ_ar[i-1])]
    
#     EOM1_sub = EOM1.subs(params).subs(past)
#     EOM2_sub = EOM2.subs(params).subs(past)
#     EOM3_sub = EOM3.subs(params).subs(past)
    
#     dU_ = sym.solve(EOM1_sub, ddx1b)
#     dW_ = sym.solve(EOM2_sub, ddz1b)
#     dQ_ = sym.solve(EOM3_sub, ddth1)
    
    var_list = [
    N_ar[i-1],D_ar[i-1],θ_ar[i-1],
    U_ar[i-1], W_ar[i-1], Q_ar[i-1],
    ddx1b, ddz1b, ddth1,
    V_, α, T_t,
    δ_e, 0,
    m_, Iy_,
    S_, b_, c_, A_, e_,
    CD_0_,
    CL_0_,CL_a_,CL_Q_,CL_de_,CL_df_,
    Cm_0_,Cm_a_,Cm_Q_,Cm_de_,Cm_df_,
    rho_, g_, np.pi   
    ]
    
    dU_ = sym.solve(lambEOM1(*var_list), ddx1b)
    dW_ = sym.solve(lambEOM2(*var_list), ddz1b)
    dQ_ = sym.solve(lambEOM3(*var_list), ddth1)
    
    try:
        Q_ar.append(Q_ar[i-1]+h*float(dQ_[0]))
    except IndexError:
        Q_ar.append(Q_ar[i-1]+h*0)
        
    try:
        U_ar.append(U_ar[i-1]+h*float(dU_[0]))
    except IndexError:
        U_ar.append(U_ar[i-1]+h*0)
        
    try:
        W_ar.append(W_ar[i-1]+h*float(dW_[0]))
    except IndexError:
        W_ar.append(W_ar[i-1]+h*0)
        
    θ_ar.append(θ_ar[i-1]+h*Q_ar[i])

    Ndot_ar.append(cos(θ_ar[i])*U_ar[i] + sin(θ_ar[i])*W_ar[i])
    Ddot_ar.append(-sin(θ_ar[i])*U_ar[i] + cos(θ_ar[i])*W_ar[i])
    
    N_ar.append(N_ar[i-1]+h*Ndot_ar[i])
    D_ar.append(D_ar[i-1]+h*Ddot_ar[i])

In [ ]:
fig1, ax1 = plt.subplots(1,1, figsize=(10,10)) #create axes

def plot_plane(i, x_in, z_in, θ_in, δ_e_in, ax):
    ax.clear(); ax.grid()
    ax.set_xlim([-50,50])
    ax.set_ylim([50,-50])
    L1 = 0.625
    L2 = 0.5
    Le = 0.125
    
    bot_x = x_in[i] - L1* cos(θ_in[i])
    bot_z = z_in[i] + L1* sin(θ_in[i])
    
    top_x = x_in[i] + L2* cos(θ_in[i])
    top_z = z_in[i] - L2* sin(θ_in[i])
    
    e_x = bot_x - Le* cos(δ_e_in+ θ_in[i])
    e_z = bot_z + Le* sin(δ_e_in+ θ_in[i])
    
    ax.plot([top_x,bot_x,e_x],[top_z,bot_z,e_z],'k')
    ax.plot(x_in[i],z_in[i],'k.')
    ax.plot(bot_x,bot_z,'.',color="Grey")
    
    ax.plot(x_in[0:i+1],z_in[0:i+1],':',color='Grey')

update = lambda i: plot_plane(i,N_ar,D_ar,θ_ar,δ_e,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(len(D_ar)),interval = 50,repeat=False)

"""name = input("Name for Video: ")
name = name + ".mp4"
print("Making video ...")
Writer = ani.writers['ffmpeg']
writer = Writer(fps=25, bitrate=1800)
animate.save(name, writer=writer)"""

HTML(animate.to_html5_video())

In [ ]:
α_t = float(trim_angles[0]) #*pi/180
δ_e = float(trim_angles[1]) #- 5*pi/180
θ_t = float(trim_angles[0])

T_t = qT*S*CDt*cos(α_t) - qT*S*CLt*sin(α_t) + m_*g_*sin(α_t)

values = [(CD_0, CD_0_),(A,A_),(e,e_),(rho,rho_),(S,S_)]
T_t = T_t.subs(values)

V_t = 18.0

data = {
    "alpha_t": α_t,
    'delta_e_t': δ_e,
    "theta_t":θ_t,
    "T_t":float(T_t),
    "V_t":V_t
}

import pickle as pkl

outfile = open('NormalTrimConditions','wb')
pkl.dump(data,outfile)
outfile.close()

In [ ]:
data